### 데이터 불러오기

- 데이터는 NE tag가 공백으로 구분되어 있다.

- *read_data* 함수를 실행하면 *file_path* 에서 텍스트 데이터를 읽고 token 리스트, tag 리스트를 반환한다.

In [3]:
import pandas as pd, numpy as np

In [4]:
df = pd.read_csv('./questions/questions_corpus_bio.csv', encoding='utf-8', index_col=0)
print(df.shape)
df.head(2)

/anaconda3/envs/keras-test/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(1678295, 3)


,question,okt,okt_tag
0,계룡시 단체석 있는 복매운탕 알아요,"['계룡시', '단체', '석', '있는', '복', '매운탕', '알아요']","['B-LOC', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'B-FOD', ..."
1,사천 마미밥상 무한 리필 사람들 후기 뭐에요,"['사천', '마미', '밥상', '무한', '리필', '사람', '들', '후기'...","['B-LOC', 'B-ORG', 'I-ORG', 'B-ADJ', 'I-ADJ', ..."


In [5]:
# 'okt', 'okt_tag' 컬럼을 list로 변경
df['okt'] = df['okt'].apply(lambda x: eval(x))
df['okt_tag'] = df['okt_tag'].apply(lambda x: eval(x))

In [6]:
df.head(2)

,question,okt,okt_tag
0,계룡시 단체석 있는 복매운탕 알아요,"[계룡시, 단체, 석, 있는, 복, 매운탕, 알아요]","[B-LOC, B-ADJ, I-ADJ, I-ADJ, B-FOD, I-FOD, O]"
1,사천 마미밥상 무한 리필 사람들 후기 뭐에요,"[사천, 마미, 밥상, 무한, 리필, 사람, 들, 후기, 뭐, 에요]","[B-LOC, B-ORG, I-ORG, B-ADJ, I-ADJ, O, O, O, O..."


데이터를 train과 test로 9:1의 비율로 나눈다.
 - *train* -> 모델 훈련;
 - *test* -> 모델 평가와 hyperparameter 조정

In [10]:
from sklearn.model_selection import train_test_split

train_tokens, test_tokens, train_tags, test_tags = \
    train_test_split(df['okt'].values
                    ,df['okt_tag'].values
                    ,test_size=0.1
                    ,random_state=42)

In [11]:
train_tokens.shape, train_tags.shape, test_tokens.shape, test_tags.shape

((1510465,), (1510465,), (167830,), (167830,))

In [12]:
train_tokens[:3]

array([list(['치맥', '타운', '사람', '들', '반응', '알려줘요']),
       list(['페리', '아', '상세', '주소', '알아보려고요']),
       list(['키친', '홀릭', '핵', '맛있는', '평', '알', '고', '계신가요'])],
      dtype=object)

In [13]:
train_tags[:3]

array([list(['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O']),
       list(['B-ORG', 'I-ORG', 'O', 'O', 'O']),
       list(['B-ORG', 'I-ORG', 'B-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O'])],
      dtype=object)

In [14]:
for i in range(0,2):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))

치맥	B-ORG
타운	I-ORG
사람	O
들	O
반응	O
알려줘요	O
페리	B-ORG
아	I-ORG
상세	O
주소	O
알아보려고요	O


### 사전 준비

신경망 모델 훈련을 위해 아래 두 가지 mapping을 사용한다:
- {token}$\to${token id}: embeddings matrix에서 해당 token의 row를 반환 
- {tag}$\to${tag id}: 신경망 출력에서 loss를 계산하기 위한 one-hot ground truth probability distribution vectors

{token or tag}$\to${index}, {tag}$\to${tag id} 두 개 딕셔너리를 반환하는 함수 *build_dict*를 정의한다.

In [15]:
from collections import defaultdict
import numpy as np

In [16]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: token / tag의 리스트
        special_tokens: 특이 token
    """
    
    # default 값이 0인 딕셔너리를 만든다.
    # (key/value를 따로 지정하지 않고, key만 저장하면 value는 0으로 자동 대입되는 딕셔너리)
    tok2idx = defaultdict(lambda: 0) 
    idx2tok = []
    
    idx = 0
    
    # special_tokens를 먼저 딕셔너리에 저장한다.
    for token in special_tokens:
        tok2idx[token] = idx
        idx2tok.append(token)
        idx += 1
    
    # tok2idx 딕셔너리에서 key값인 token or tag는 unique 해야 한다.
    # 따라서 tokens_or_tags에서 special_tokens에 해당하지 않는 tokens_or_tags만 새로 index를 준다.
    for token_list in tokens_or_tags:
        for token in token_list:
            if token not in tok2idx:
                tok2idx[token] = idx
                idx2tok.append(token)
                idx += 1
    
    return tok2idx, idx2tok

*build_dict* 함수를 이용하여 token과 tag의 딕셔너리를 만든다. 여기서 special_tokens는 아래와 같다.
 - `<UNK>` 언어 토큰에 해당하지 않는 것 (unknown)
 - `<PAD>` 문장에 대한 batch를 생성할 때 부여할 문장의 길이와 같은 padding 토큰

In [17]:
special_tokens = ['<UNK>', '<PAD>'] # unknown, padding
special_tags = ['O'] # out of tag

# 딕셔너리 생성 
token2idx, idx2token = build_dict(train_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

In [18]:
token2idx

defaultdict(<function __main__.build_dict.<locals>.<lambda>()>,
            {'<UNK>': 0,
             '<PAD>': 1,
             '치맥': 2,
             '타운': 3,
             '사람': 4,
             '들': 5,
             '반응': 6,
             '알려줘요': 7,
             '페리': 8,
             '아': 9,
             '상세': 10,
             '주소': 11,
             '알아보려고요': 12,
             '키친': 13,
             '홀릭': 14,
             '핵': 15,
             '맛있는': 16,
             '평': 17,
             '알': 18,
             '고': 19,
             '계신가요': 20,
             '고색': 21,
             '소머리국밥': 22,
             '냉면': 23,
             '가성': 24,
             '비': 25,
             '좋은': 26,
             '솔직': 27,
             '후기': 28,
             '부탁': 29,
             '해': 30,
             '이벤트': 31,
             '홀': 32,
             '있는': 33,
             '하모': 34,
             '사시미': 35,
             '지리는': 36,
             '곳': 37,
             '뭐': 38,
             '였어요': 39,
             '통

In [19]:
idx2token

['<UNK>',
 '<PAD>',
 '치맥',
 '타운',
 '사람',
 '들',
 '반응',
 '알려줘요',
 '페리',
 '아',
 '상세',
 '주소',
 '알아보려고요',
 '키친',
 '홀릭',
 '핵',
 '맛있는',
 '평',
 '알',
 '고',
 '계신가요',
 '고색',
 '소머리국밥',
 '냉면',
 '가성',
 '비',
 '좋은',
 '솔직',
 '후기',
 '부탁',
 '해',
 '이벤트',
 '홀',
 '있는',
 '하모',
 '사시미',
 '지리는',
 '곳',
 '뭐',
 '였어요',
 '통영',
 '전망',
 '멋있는',
 '도루묵',
 '추천',
 '해줄만',
 '한',
 '데',
 '좀',
 '각설탕',
 '알려주시겠어요',
 '이쁜',
 '샷',
 '대박',
 '맛집',
 '있아요',
 '경기도',
 '이천시',
 '향',
 '긋한',
 '올갱이',
 '알려주세요',
 '강서',
 '티',
 '하우스',
 '이',
 '정말',
 '끝내주는',
 '뭔',
 '데예',
 '돈부리',
 '야',
 '리뷰',
 '뭔가',
 '요',
 '울산',
 '동구',
 '양념',
 '잘',
 '배',
 '인',
 '오지',
 '는',
 '석쇠',
 '마루',
 '어디',
 '인데',
 '경상북도',
 '예천군',
 '밥',
 '반찬',
 '예요',
 '솟구쳐',
 '차기',
 '여쭤',
 '보려구요',
 '충청남도',
 '금산군',
 '다져',
 '주는',
 '쩌',
 '물어볼라는데요',
 '서비스',
 '많이',
 '반',
 '쎄오',
 '였죠',
 '저렴하면서',
 '양',
 '많은',
 '꼬리곰탕',
 '해줘',
 '부천시',
 '청탁',
 '씹는',
 '식감',
 '보려고요',
 '장관',
 '순',
 '살',
 '치킨',
 '뭐드',
 '라',
 '아리수',
 '자세한',
 '알려주나요',
 '서울',
 '연탄',
 '갈비',
 '할',
 '있으신',
 '가요',
 '행복',
 '마을',
 '눅눅하지',
 '않은',
 '부탁드려요'

In [20]:
tag2idx

defaultdict(<function __main__.build_dict.<locals>.<lambda>()>,
            {'O': 0,
             'B-ORG': 1,
             'I-ORG': 2,
             'B-ADJ': 3,
             'I-ADJ': 4,
             'B-FOD': 5,
             'I-FOD': 6,
             'B-LOC': 7,
             'I-LOC': 8})

In [21]:
idx2tag

['O', 'B-ORG', 'I-ORG', 'B-ADJ', 'I-ADJ', 'B-FOD', 'I-FOD', 'B-LOC', 'I-LOC']

token <-> index, index <-> tag 서로 변환해주는 함수를 정의한다.

In [22]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

In [1]:
import keras

Using TensorFlow backend.


In [2]:
keras.__version__

'2.2.4'

In [24]:
train_tokens,train_tags

(array([list(['치맥', '타운', '사람', '들', '반응', '알려줘요']),
        list(['페리', '아', '상세', '주소', '알아보려고요']),
        list(['키친', '홀릭', '핵', '맛있는', '평', '알', '고', '계신가요']), ...,
        list(['당진', '순이', '네', '밥집', '솔직', '후기', '궁금합니다']),
        list(['폭포', '있는', '환상', '적', '인', '추천', '할', '데', '어디', '있죠']),
        list(['청도군', '서비스', '는', '덤', '인', '스페인', '좋은', '데', '어디', '인데', '요'])],
       dtype=object), array([list(['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O']),
        list(['B-ORG', 'I-ORG', 'O', 'O', 'O']),
        list(['B-ORG', 'I-ORG', 'B-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O']),
        ..., list(['B-LOC', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O']),
        list(['B-ADJ', 'I-ADJ', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O', 'O']),
        list(['B-LOC', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'I-ADJ', 'B-FOD', 'O', 'O', 'O', 'O', 'O'])],
       dtype=object))

In [78]:
from gensim.models import word2vec

In [80]:
embedding = word2vec.Word2Vec(X_train)

In [85]:
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
model = KeyedVectors.load_word2vec_format('my.embedding', binary=False, encoding='utf-8')

In [25]:
from anago.tagger import Tagger
from anago.trainer import Trainer
from anago.wrapper import Sequence
from anago.utils import NERSequence

/anaconda3/envs/keras-test/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [27]:
model = Sequence()

In [28]:
model.fit(x_train=['hey','there','im','test']
        ,y_train=['O','O','O','O'])

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
model = Sequence.fit

### Batch 생성

신경망 모델은 batch를 이용하여 훈련된다. 즉, 신경망의 가중치는 매번 일정한 순서를 따라 조정된다는 뜻이다. 여기서 중요한 것은 batch 내의 모든 순서가 동일한 길이를 가져야 한다는 점이다. 이를 위해 `<PAD>` 토큰을 이용해 padding을 한다. 아래 함수 *batches_generator* 를 이용해 신경망 모델에 이용할 batch를 생성한다.

In [45]:
import tensorflow as tf
import numpy as np

In [46]:
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    """
        tokens와 tags 각각에 padding된 batch를 생성한다.
    """
    
    n_samples = len(tokens)

    if shuffle:
        # shuffle=True일 경우, n_samples를 랜덤으로 섞는다.
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)
    
    # n_samples를 batch_size로 나눈 몫을 n_batches에 저장한다.
    n_batches = n_samples // batch_size

    # allow_smaller_last_batch가 True고, 
    # n_samples를 batch_size로 나눈 나머지가 존재하면,
    # n_batches에 1을 추가한다.
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1
    
    # n_batchs 수 만큼 batch를 생성한다.
    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start

        x_list = []
        y_list = []
        max_len_token = 0

        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx])) # tokens 리스트를 index 리스트로 바꿔서 x_list에 저장
            y_list.append(tags2idxs(tags[idx])) # tags 리스트를 index 리스트로 바꿔서 y_list에 저장
            max_len_token = max(max_len_token, len(tags[idx])) # 현재까지 batch에서 가장 큰 tag리스트 사이즈를 저장
            
        # padding 인덱스로 채워진 nd-array를 생성한다.
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        
        # 0으로 채워진 (current_batch_size,) nd-array를 생성한다.
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            lengths[n] = utt_len
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths

## RNN (Recurrent Neural Network) 모델 생성

LSTM을 사용해 각 문장의 tag와 token의 확률 분포를 알아내는 모델을 생성한다. 여기서 문장 속의 각 tag/token의 앞뒤 문맥을 모두 고려하기 위해 Bi-Directional LSTM (Bi-LSTM)을 사용한다. tag 분류를 위한 Dense layer는 가장 상위에서 사용된다.

In [65]:
tf.version

<module 'tensorflow_core._api.v2.version' from '/anaconda3/envs/keras-test/lib/python3.6/site-packages/tensorflow_core/_api/v2/version/__init__.py'>

In [62]:
class BiLSTMModel():
    
    def declare_placeholders(self):
        """
            모델에 사용할 placeholder를 지정한다.
        """ 
        # input과 ground truth output의 Placeholders
        self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
        self.ground_truth_tags = tf.placeholder(dtype=tf.int32, shape=[None, None], name='ground_truth_tags')

        # sequence의 length의 Placeholder
        self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name='lengths') 

        # dropout keep probability의 Placeholder
        # 1.0을 디폴트 값으로 준다.
        self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])

        # 학습률의 Placeholder
        self.learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[], name='learning_rate_ph')

    
    def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
        """
            bi-LSTM 구조 설계하고 logit을 계산
        """
        # embedding 변수 생성
        initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
        embedding_matrix_variable = tf.Variable(initial_embedding_matrix, name='embeddings_matrix', dtype=tf.float32)

        # 은닉층의 RNN cell을 생성
        # dropout을 생성하고 dropout placeholder 값을 대입
        forward_cell =  tf.nn.rnn_cell.DropoutWrapper(
            tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn, forget_bias=3.0),
            input_keep_prob=self.dropout_ph,
            output_keep_prob=self.dropout_ph,
            state_keep_prob=self.dropout_ph
        )
        backward_cell = tf.nn.rnn_cell.DropoutWrapper(
            tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn, forget_bias=3.0),
            input_keep_prob=self.dropout_ph,
            output_keep_prob=self.dropout_ph,
            state_keep_prob=self.dropout_ph
        )

        # self.input_batch의 Look up embeddings 생성
        # Shape: [batch_size, sequence_len, embedding_dim].
        embeddings = tf.nn.embedding_lookup(embedding_matrix_variable, self.input_batch)

        # Bidirectional Dynamic RNN 에 통과
        # Shape: [batch_size, sequence_len, 2 * n_hidden_rnn]. 
        (rnn_output_fw, rnn_output_bw), _ =  tf.nn.bidirectional_dynamic_rnn(
                                                  cell_fw= forward_cell
                                                , cell_bw= backward_cell
                                                , dtype=tf.float32
                                                , inputs=embeddings
                                                , sequence_length=self.lengths
                                            )
        rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)

        # 상위에 Dense layer 생성
        # Shape: [batch_size, sequence_len, n_tags].   
        self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)
        
    def compute_predictions(self):
        """
            logit을 확률로 transform하여 tag를 예측
        """
        # softmax 함수 생성
        softmax_output = tf.nn.softmax(self.logits)

        # argmax 를 이용하여 tag를 예측
        self.predictions = tf.argmax(softmax_output, axis = -1)
        
    def compute_loss(self, n_tags, PAD_index):
        """
            logit을 이용해 cross-entopy loss를 계산
        """
        # cross entropy function 생성
        ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
        loss_tensor = tf.nn.softmax_cross_entropy_with_logits(
            labels=ground_truth_tags_one_hot
            , logits=self.logits
        )

        mask = tf.cast(tf.not_equal(self.input_batch, PAD_index), tf.float32)
        # <PAD> token 은 제외하고 loss를 계산하는 loss function 생성
        self.loss = tf.reduce_mean(tf.reduce_sum(tf.multiply(loss_tensor, mask), axis=-1) \
                                   / tf.reduce_sum(mask, axis=-1))
        
    def perform_optimization(self):
        """
            모델의 optimizer, train_op 지정
        """
        # optimizer 생성
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate_ph)
        self.grads_and_vars = self.optimizer.compute_gradients(self.loss)

        # self.grads_and_vars에 Gradient clipping 적용
        clip_norm = tf.cast(1.0, tf.float32)
        self.grads_and_vars = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in self.grads_and_vars]
        self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)
    
    def __init__(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
        """
            모델 생성 시, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index를 지정하는 init 메소드
        """
        self.declare_placeholders()
        self.build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
        self.compute_predictions()
        self.compute_loss(n_tags, PAD_index)
        self.perform_optimization()
        
    def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
        feed_dict = {self.input_batch: x_batch,
                     self.ground_truth_tags: y_batch,
                     self.learning_rate_ph: learning_rate,
                     self.dropout_ph: dropout_keep_probability,
                     self.lengths: lengths}

        session.run(self.train_op, feed_dict=feed_dict)
        
    def predict_for_batch(self, session, x_batch, lengths):
        predictions = session.run(self.predictions
                                 , feed_dict={self.input_batch:x_batch, self.lengths:lengths})
        return predictions

# 위 클래스 정의했으면 아래로 쭉! >> *모델평가*로 직행하세요

### BiLSTMModel 클래스 각각의 함수 설명

신경망이 실행되는 동안 입력할 data를 지정하기 위해 아래와 같은 [placeholders](https://www.tensorflow.org/versions/master/api_docs/python/tf/placeholder) 를 생성한다.

 - *input_batch* — 단어 sequence (shape : [batch_size, sequence_len]);
 - *ground_truth_tags* — 태그 sequence (shape :  [batch_size, sequence_len]);
 - *lengths* — 패딩 되지 않은 sequence의 길이 (shape :  [batch_size]);
 - *dropout_ph* — dropout keep probability; 
 - *learning_rate_ph* — 학습률; 

아래 함수에서 shape를 지정하기 위해 *None* 값을 주었다. data (단어, 태그) 로 어떤 size든 올 수 있다는 뜻이다.

In [16]:
def declare_placeholders(self):
    """
        모델에 사용할 placeholder를 지정한다.
    """

    # input과 ground truth output의 Placeholders
    self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
    self.ground_truth_tags = tf.placeholder(dtype=tf.int32, shape=[None, None], name='ground_truth_tags')
  
    # sequence의 length의 Placeholder
    self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name='lengths') 
    
    # dropout keep probability의 Placeholder
    # 1.0을 디폴트 값으로 준다.
    self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    
    # 학습률의 Placeholder
    self.learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[], name='learning_rate_ph')

In [17]:
BiLSTMModel.__declare_placeholders = classmethod(declare_placeholders)

#### 신경망의 layer을 아래와 같이 지정한다.

- [tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)를 이용하여 embedding matrix (*embeddings_matrix*)를 생성하고 랜덤 int값을 대입한다.
- forward LSTM / backward LSTM cell을 각각 생성한다. TensorFlow의 *LSTMCell* 메소드를 이용한다. 
    - (다른 타입의 cell에 대한 설명은 다음을 참조 : e.g. GRU cells. [This](http://colah.github.io/posts/2015-08-Understanding-LSTMs/))
- [DropoutWrapper](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper) 를 이용해 cell을 wrapping 한다. Dropout 은 신경망 모델의 regularization 에서 중요한 역할을 한다. 위의 `declare_placeholders`에서 지정한 `dropout_ph`를 이용한다.

#### input_batch를 변환하여 computation graph를 아래와 같이 생성한다:

- [Look up](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) 메소드를 이용해 embeddings를 생성한다.
- embeddings 를 [Bidirectional Dynamic RNN](https://www.tensorflow.org/api_docs/python/tf/nn/bidirectional_dynamic_rnn) 에 적용한다. 위의  `declare_placeholders`에서 지정한 `lengths` placeholder를 이용해 padding token은 RNN에서 계산되지 않도록 한다.
- 가장 상위에 dense layer를 생성한다. 이 layer의 output은 바로 loss function에 입력된다.

In [0]:
def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
    """
        bi-LSTM 구조 설계하고 logit을 계산
    """
    
    # embedding 변수 생성
    initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
    embedding_matrix_variable = tf.Variable(initial_embedding_matrix, name='embeddings_matrix', dtype=tf.float32)
    
    # 은닉층의 RNN cell을 생성
    # dropout을 생성하고 dropout placeholder 값을 대입
    forward_cell =  tf.nn.rnn_cell.DropoutWrapper(
        tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn, forget_bias=3.0),
        input_keep_prob=self.dropout_ph,
        output_keep_prob=self.dropout_ph,
        state_keep_prob=self.dropout_ph
    )
    backward_cell = tf.nn.rnn_cell.DropoutWrapper(
        tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn, forget_bias=3.0),
        input_keep_prob=self.dropout_ph,
        output_keep_prob=self.dropout_ph,
        state_keep_prob=self.dropout_ph
    )

    # self.input_batch의 Look up embeddings 생성
    # Shape: [batch_size, sequence_len, embedding_dim].
    embeddings = tf.nn.embedding_lookup(embedding_matrix_variable, self.input_batch)
    
    # Bidirectional Dynamic RNN 에 통과
    # Shape: [batch_size, sequence_len, 2 * n_hidden_rnn]. 
    (rnn_output_fw, rnn_output_bw), _ =  tf.nn.bidirectional_dynamic_rnn(
                                              cell_fw= forward_cell
                                            , cell_bw= backward_cell
                                            , dtype=tf.float32
                                            , inputs=embeddings
                                            , sequence_length=self.lengths
                                        )
    rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)

    # 상위에 Dense layer 생성
    # Shape: [batch_size, sequence_len, n_tags].   
    self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)

In [0]:
BiLSTMModel.__build_layers = classmethod(build_layers)

신경망에서 예측값을 나타내기 위해서는 [softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) 를 마지막 layer로 적용하여 tag 값을 나타내도록 해야 한다. [argmax](https://www.tensorflow.org/api_docs/python/tf/argmax).

In [0]:
def compute_predictions(self):
    """
        logit을 확률로 transform하여 tag를 예측
    """
    
    # softmax 함수 생성
    softmax_output = tf.nn.softmax(self.logits)
    
    # argmax 를 이용하여 tag를 예측
    self.predictions = tf.argmax(softmax_output, axis = -1)

In [0]:
BiLSTMModel.__compute_predictions = classmethod(compute_predictions)

모델 training 중에는 loss function이 필요하다. 이 모델에서는 loss function으로 [cross-entropy loss](http://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html#cross-entropy)를 사용한다. 

TensorFlow에는 [cross entropy with logits](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits_v2) 메소드로 구현되어 있다. 이 메소드는 모델의 softmax의 확률이 아닌, logit에 적용되어야 한다. 또한, `<PAD>` 토큰의 loss는 계산되지 않아야 한다. 따라서 [mean](https://www.tensorflow.org/api_docs/python/tf/reduce_mean) 을 계산하기 전에 `<PAD>`의 토큰은 제거되어야 한다.

In [0]:
def compute_loss(self, n_tags, PAD_index):
    """
        logit을 이용해 cross-entopy loss를 계산
    """
    
    # cross entropy function 생성
    ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
    loss_tensor = tf.nn.softmax_cross_entropy_with_logits(
        labels=ground_truth_tags_one_hot
        , logits=self.logits
    )
    
    mask = tf.cast(tf.not_equal(self.input_batch, PAD_index), tf.float32)
    # <PAD> token 은 제외하고 loss를 계산하는 loss function 생성
    self.loss = tf.reduce_mean(tf.reduce_sum(tf.multiply(loss_tensor, mask), axis=-1) / tf.reduce_sum(mask, axis=-1))

In [0]:
BiLSTMModel.__compute_loss = classmethod(compute_loss)

마지막으로 지정할 것은 loss를 optimize하는 방법이다. 이를 위해서 TensorFlow의 [Adam](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer) optimizer 를 사용한다. (학습률은 위에서 지정한 learning_rate_ph를 사용한다. 또한 발산하는 gradients를 제거하기 위해서 [clip_by_norm](https://www.tensorflow.org/api_docs/python/tf/clip_by_norm) 함수를 이용해 clipping을 적용한다.

In [0]:
def perform_optimization(self):
    """
        모델의 optimizer, train_op 지정
    """
    
    # optimizer 생성
    self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate_ph)
    self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
    
    # self.grads_and_vars에 Gradient clipping 적용
    clip_norm = tf.cast(1.0, tf.float32)
    self.grads_and_vars = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in self.grads_and_vars]
    self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)

In [0]:
BiLSTMModel.__perform_optimization = classmethod(perform_optimization)

In [0]:
def init_model(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
    self.__declare_placeholders()
    self.__build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
    self.__compute_predictions()
    self.__compute_loss(n_tags, PAD_index)
    self.__perform_optimization()

In [0]:
BiLSTMModel.__init__ = classmethod(init_model)

## 신경망 Training & tag 예측

[Session.run](https://www.tensorflow.org/api_docs/python/tf/Session#run) 메소드는 위에서 정의한 graph를 초기화하는 메소드이다. 신경망을 training 하기 위해서 *perform_optimization* 메소드 안의 *self.train_op*을 먼저 계산한다. tag를 예측하기 위해서는 *self.predictions*을 계산하면 된다. 

In [0]:
def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
    feed_dict = {self.input_batch: x_batch,
                 self.ground_truth_tags: y_batch,
                 self.learning_rate_ph: learning_rate,
                 self.dropout_ph: dropout_keep_probability,
                 self.lengths: lengths}
    
    session.run(self.train_op, feed_dict=feed_dict)

In [0]:
BiLSTMModel.train_on_batch = classmethod(train_on_batch)

*feed_dict* 를 *x_batch* 와 *lengths* 를 이용해 초기화한 후, *predict_for_batch* 함수를 이용해 *prediction* 을 실행한다.

In [0]:
def predict_for_batch(self, session, x_batch, lengths):
    predictions = session.run(self.predictions
                             , feed_dict={self.input_batch:x_batch, self.lengths:lengths})
    return predictions

In [0]:
BiLSTMModel.predict_for_batch = classmethod(predict_for_batch)

## 모델 평가
 - *predict_tags*: 모델을 이용해 예측한 token, tag를 반환한다;
 - *eval_conll*: 결과의 precision, recall, F1 score를 계산한다.

In [56]:
from evaluation import precision_recall_f1

ModuleNotFoundError: No module named 'evaluation'

In [57]:
def predict_tags(model, session, token_idxs_batch, lengths):
    """
        예측한 결과를 token, tag 형태로 반환
    """
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    """
        Computes NER quality measures using CONLL shared task script.
    """
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

## Run your experiment

Create *BiLSTMModel* model with the following parameters:
 - *vocabulary_size* — token의 수;
 - *n_tags* — tag의 수;
 - *embedding_dim* — embedding의 차원, recommended value: 200;
 - *n_hidden_rnn* — RNN 은닉층 size, recommended value: 200;
 - *PAD_index* — padding token (`<PAD>`)의 index.

hyperparameters 세팅:
- *batch_size*: 32;
- 4 epochs;
- *learning_rate* 시작값: 0.005
- *learning_rate_decay*: sqrt(2);
- *dropout_keep_probability*: 0.1, 0.5, 0.9.

In [63]:
model = BiLSTMModel(20505, 21, 200, 200
                    , token2idx['<PAD>'])

batch_size = 32
n_epochs = 4
learning_rate = 0.005
learning_rate_decay = np.sqrt(2)

#0.1, 0.5, 0.9
dropout_keep_probability = 0.5


AttributeError: module 'tensorflow' has no attribute 'placeholder'

If you got an error *"Tensor conversion requested dtype float64 for Tensor with dtype float32"* in this point, check if there are variables without dtype initialised. Set the value of dtype equals to *tf.float32* for such variables.

In [91]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Start training... \n')
for epoch in range(n_epochs):
    # For each epoch evaluate the model on train and validation data
    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(n_epochs) + '-' * 20)
    print('Train data evaluation:')
    eval_conll(model, sess, train_tokens, train_tags, short_report=True)
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_tokens, validation_tags, short_report=True)
    
    # Train the model
    for x_batch, y_batch, lengths in batches_generator(batch_size, train_tokens, train_tags):
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability)
        
    # Decaying the learning rate
    learning_rate = learning_rate / learning_rate_decay
    
print('...training finished.')

Start training... 

-------------------- Epoch 1 of 4 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 67843 phrases; correct: 130.

precision:  0.19%; recall:  2.90%; F1:  0.36

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 8183 phrases; correct: 24.

precision:  0.29%; recall:  4.47%; F1:  0.55

-------------------- Epoch 2 of 4 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 2104 phrases; correct: 524.

precision:  24.90%; recall:  11.67%; F1:  15.90

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 192 phrases; correct: 48.

precision:  25.00%; recall:  8.94%; F1:  13.17

-------------------- Epoch 3 of 4 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 4673 phrases; correct: 1888.

precision:  40.40%; recall:  42.06%; F1:  41.21

Validation data evaluation:
processed 12836 tokens with 537 ph

Now let us see full quality reports for the final model on train, validation, and test sets. To give you a hint whether you have implemented everything correctly, you might expect F-score about 40% on the validation set.

**The output of the cell below (as well as the output of all the other cells) should be present in the notebook for peer2peer review!**

In [92]:
print('-' * 20 + ' Train set quality: ' + '-' * 20)
train_results = eval_conll(model, sess, train_tokens, train_tags, short_report=False)
    
print('-' * 20 + ' Validation set quality: ' + '-' * 20)
validation_results = eval_conll(model, sess, validation_tokens, validation_tags, short_report=False)

print('-' * 20 + ' Test set quality: ' + '-' * 20)
test_results = eval_conll(model, sess, test_tokens, test_tags, short_report=False)

-------------------- Train set quality: --------------------
processed 105778 tokens with 4489 phrases; found: 4824 phrases; correct: 3346.

precision:  69.36%; recall:  74.54%; F1:  71.86

	     company: precision:   74.66%; recall:   85.69%; F1:   79.80; predicted:   738

	    facility: precision:   60.27%; recall:   71.02%; F1:   65.20; predicted:   370

	     geo-loc: precision:   75.40%; recall:   93.57%; F1:   83.51; predicted:  1236

	       movie: precision:   14.29%; recall:    1.47%; F1:    2.67; predicted:     7

	 musicartist: precision:   43.37%; recall:   46.55%; F1:   44.91; predicted:   249

	       other: precision:   58.60%; recall:   85.07%; F1:   69.40; predicted:  1099

	      person: precision:   84.50%; recall:   86.12%; F1:   85.30; predicted:   903

	     product: precision:   55.25%; recall:   38.05%; F1:   45.07; predicted:   219

	  sportsteam: precision:  100.00%; recall:    1.38%; F1:    2.73; predicted:     3

	      tvshow: precision:    0.00%; recall:  

### Conclusions

Could we say that our model is state of the art and the results are acceptable for the task? Definately, we can say so. Nowadays, Bi-LSTM is one of the state of the art approaches for solving NER problem and it outperforms other classical methods. Despite the fact that we used small training corpora (in comparison with usual sizes of corpora in Deep Learning), our results are quite good. In addition, in this task there are many possible named entities and for some of them we have only several dozens of trainig examples, which is definately small. However, the implemented model outperforms classical CRFs for this task. Even better results could be obtained by some combinations of several types of methods, e.g. see [this](https://arxiv.org/abs/1603.01354) paper if you are interested.